# data_cleansing_tutorial
Data cleansing tutorial in Pandas for Chipy Scientific SIG

## Introduction
Data cleansing is an important part of the data analysis / data science workflow because in many cases, these practices require a very clean and standardized data set. Unfortunately, in many cases raw data sets usually contain potential errors such as containing missing values, coming in different file formats, redundancies, inconsistent naming, incorrect data types, just plain wrong values, amongst many problems. All of these concerns should be addressed in the data cleansing phase so that subsequent analysis proceeds smoothly and repeatably.

## Goals
### Goals for this tutorial
- Introduce some commonly used data cleansing techniques in Pandas including:
    - Data import from CSV (excel and stuff to come later)
    - Pandas Series and DataFrames
    - Simple exploration and summaries of data
    - Selecting data
    - Adding and removing data
    - Removing nulls
    - Fixing data types
    - Sorting
    - Concatenation and joining
    - Aggregations and group operations
    - Simple plotting (more to come in a separate tutorial)

### Goals for this example study
With all the media about vaccines causing autism spiking ~2009, we want to explore the rates of vaccinations over the past few years.

For this tutorial, we will focus on Polio in the state of Illinois. Some potential end goals are:

- Has there been an overall decrease in the vaccination rates?
    - Looking at data from years ending in 2005, 2010, and 2015
- What schools decreased the most from 2005 to 2015?
- What vaccines decreased the most from 2005 to 2015?
- OPTIONAL: What schools are at the highest risk of decreasing further?

## Dataset information
Our dataset will come from the Illinois State Board of Education. The dataset is the Immuniation School Survey from the Health Requirements/Student Health Data page. The link is: http://www.isbe.net/research/htmls/immunization.htm 

## Methodology (important)
In order to get a quick sense of the impact of the media coverage ~2009, we decided to pull data from 3 years: 2004-2005, 2009-2010, and 2014-2015.

To measure the impact, we will be looking at the percentage of immunized kids to see if the media decreased (or increased?) that amount at all. This percentage will be **number of kids immunized at the school / total kids enrolled at the school**.

## Import packages

For this tutorial we'll be using Pandas, so let's import that now. In addition, we usually also import Numpy (for other numerical operations) and Matplotlib (for easy plotting).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# this line makes the plots appear in the notebook
%matplotlib inline

## Data Format, Import, and Verification

Before we imported the data, we should note that we first pre-processed the raw data (Excel files from the website) by exporting the relevant sheets into CSV. We will now load these CSV files into Pandas.

The reason we use CSV is because it is an very commonly used between file format that can be transitioned to and from many different software platforms.

### Years 04-05

Lets begin by reading the school year of 2004-2005.

In [ ]:
df04 = pd.read_csv('../data/immunization_04-05.csv')

The *head()* method is a quick way to see the first 5 rows.

In [ ]:
df04.head()

Here we see that the above header information was imported incorrectly. The correct headers were imported into row 0. Because we are lazy, we will re-import the data and drop the first row instead of formatting this current version.

In [ ]:
# use the skiprows argument here because we don't need the first row
df04 = pd.read_csv('../data/immunization_04-05.csv', skiprows=1)
df04.head()

Cool, the columns are correct but per our goals we only need the following columns:
1. school distict
2. school id
3. total enrollment
4. polio vaccinations

One easy way to do this is to simply select the columns we want. This is done using the *ix[]* indexing function. This should be very familiar for people who used R.

In [ ]:
# name the columns we want to keep, the only select those columns
columns_to_keep = ['RCDT', 'School', 'Total Sch. Enroll.', 'Polio Prot.']
df04 = df04.ix[:, columns_to_keep] # takes all rows, takes only the columns in the array columns_to_keep
df04.head()

Interesting, the first row has NaN. Looking back at the table above, this is because the original table has some extra rows for Counties e.g. "Adams County".

Let's simply drop the NaNs in this table using the *dropna()* method.

In [ ]:
df04.dropna(inplace=True) # inplace=True replaces the previous data frame.
                          # This is the same as df04 = df04.dropna(inplace=False)
df04.head()

#### Summary Statistics

Instead of looking just the first 5 rows, let's check out some summary statistics. This is done using the *describe()* method.

In [ ]:
df04.describe()

We should also make take a look at the datatypes (e.g. string, number) of these columns using *dtypes()*.

In [ ]:
df04.dtypes

Uh oh, the two columns are supposed to be numerical, not strings (denoted here as objects).

Let's convert these now. To convert an entire column, we use the *applymap()* method. This essentially takes each value of the column and applies the function. In our case, the function is to change the datatype to int.

In [ ]:
# inplace isn't available for this function, thus asign manually
cols = ['Total Sch. Enroll.', 'Polio Prot.']
df04[cols] = df04[cols].applymap(lambda x: int(x))
df04.head()

This error shows that there's are values with commas, as a result Python can't automatically change it into an int.

We have to remove the comma, then convert to int. We do this using the *replace()* method.

In [ ]:
# same applymap() above, but our function replaces the string then converts
df04[cols] = df04[cols].applymap(lambda x: int(x.replace(',', '')))
df04.head()

Try looking at the datatypes again.

In [ ]:
df04.dtypes

Beautiful. Now let's run those summary stats again.

In [ ]:
df04.describe(include='all')

This looks fine so far.

We can probably do the same things to the next datasets.

### Years 09-10

Now that the 2004-2005 data is out of the way, let's continue to 2009-2010. We expect to be using the same methods and techniques as above.

Import again.

In [ ]:
df09 = pd.read_csv('../data/immunization_09-10.csv')
df09.head()

Same header issue as before. Same fix as above.

In [ ]:
df09 = pd.read_csv('../data/immunization_09-10.csv', skiprows=1)
df09.head()

Looks good, now we want just the polio columns again.

In [ ]:
# name the columns we want to keep, the only select those columns
columns_to_keep = ['RCDT', 'School', 'Total Sch. Enroll.', 'Polio Prot.']
df09.ix[:, columns_to_keep].head() # takes all rows, takes only the columns in the array columns_to_keep

Oh, this is because the columns are named differently than from the 04-05 dataset! We need to get the correct column names for this table. Let's see the column names first using the *columns* attribute.

In [ ]:
df09.columns

In [ ]:
# again, select the columns we want and only take those
columns_to_keep = ['RCDTS', 'Enrollment', 'PolioProt']
df09 = df09[columns_to_keep]
# might as well remove NAs too.
df09.dropna(inplace=True)
df09.head()

Let's make a quick note here that in this dataset, school district and school ID were combined into a single column.

In [ ]:
df09.dtypes

How nice, they're floats already.

In [ ]:
df09.describe(include='all')

Looks fine too.

Let's go to the final dataset.

### Years 14-15

In [ ]:
df14 = pd.read_csv('../data/Immunization_14-15.csv')
df14.head()

There are header issues again, but this seems different. Looks like we want RCDTS from row 0, and the rest from row 1.

Instead of what we did before, let's modify this by manually taking the raw data we want, then renaming the columns.

In [ ]:
# we want rows from 2 to the end, we want columns 0, 3, and 8
df14 = df14.ix[2:, [0, 3, 8]]
df14.dropna(inplace=True)
# now customize and set the names we want for the columns
new_column_names = ['RCDTS', 'Enrolled', 'Protected']
df14.columns = new_column_names
df14.head()

So far so good. Lets run the summary stats again.

In [ ]:
df14.describe()

In [ ]:
df14.dtypes

Objects again...

In [ ]:
cols = ['Enrolled', 'Protected']
df14[cols] = df14[cols].applymap(lambda x: int(x.replace(',', '')))
df14.head()

In [ ]:
df14.describe(include='all')

That max looks crazy. What RCDTS does it belong to?

In [ ]:
df14.sort(columns='Enrolled', ascending=False).head()

That's not a valid RCDTS... Looks like we can just dump the row with id=4876 using the *drop()* method.

In [ ]:
df14 = df14.drop(4876, axis=0)

In [ ]:
df14.describe(include='all')

Much better.

And that should be the last fix for our data import.

## Data Concatenation

Next we will concatenate the tables to create a single table that will contain all the necessary data for our analysis.

We noticed from above that all of the tables have different columns names. This will have to be fixed so that the tables can be joined easily.

Tables for 14-15 and 09-10 both only have 3 columns so let's merge those first.

However, to keep track of where the data comes from, let's add a "Year" column to both.

Adding a column is similar to adding a new field and dictionary i.e. using square brackets.

In [ ]:
# add year column with correct values
df14['Year'] = 2014
df14.head()

In [ ]:
df09['Year'] = 2009
df09.head()

Now we rename the columns so that the DataFrames can be easily concatenated.

In [ ]:
# rename columns
columns = ['rcdts', 'enrollment', 'protected', 'year']
df09.columns = columns
df14.columns = columns

In [ ]:
# concatenate
df = pd.concat([df14, df09])
print df[:5] # print first 5, same functionally as head()
print df[-5:] # print last 5

Great, looks like both our datasets are here then.

Time to join the table from year 04-05.

However, we remember that year 04-05 has 4 columns because school is broken out:

In [ ]:
df04.head()

Another thing we notice is that the years 04-05 have 'n/a' values for School... this poses a problem for future analyses. Without being sure of the School values, we're not comfortable comparing this year with the other 2 years.

After much debate, we decided to remove school completely from our current analysis.

Unfortunately, this means we will have to remove the School values from our other datasets.

In [ ]:
df.insert(1, 'rcdt', 0)
df['rcdt'] = df['rcdts'].apply(lambda x: x[:-4]) # use apply here because Series does not have an applymap() method
df.head()

Now that we have the RCDT column, we can drop the origina RCDTS column.

In [ ]:
# drop old rcdts column
df.drop('rcdts', axis=1, inplace=True)
df.head()

Now with the master dataset prepped, let's clean up the 04-05 dataset.

In [ ]:
columns_we_want = ['rcdt', 'enrollment', 'protected', 'year']

In [ ]:
# drop the school column
df04.drop('School', axis=1, inplace=True)
# add the year so we remember where this dataset came from
df04['year'] = 2004
# now rename the columns
df04.columns = columns_we_want
df04.head()

Finally, let's concatenate this dataset into the master dataset, again using the *concat()* method.

In [ ]:
df = pd.concat([df, df04])
print df[:5] # print first 5, same functionally as head()
print df[-5:] # print last 5

## Optional: Data Analysis

The original goal of this analysis is to see if the protected percentage dropped over the years. Let's run this analysis, which shouldn't take long now that we have the data in the way we want it.

In [ ]:
df_year = df.groupby('year').aggregate({'enrollment':'sum', 'protected':'sum'})
df_year.head()

In [ ]:
df_year['prot_perc'] = df_year['protected']/df_year['enrollment']
df_year.head()

In [ ]:
%matplotlib inline
df_year.prot_perc.plot()

Looks like the rate of immunization has dropped a little over these years. However, with only 3 data points we can't be too certain. Best if we were to take the other years in between into account. That's for another time though.